In [1]:
!pwd

/content


In [2]:
%cd /content/drive/My\ Drive/Text_Summarization_Code

/content/drive/My Drive/Text_Summarization_Code


In [3]:
!pwd

/content/drive/My Drive/Text_Summarization_Code


In [4]:
%tensorflow_version 1.x
import tensorflow as tf
tf.__version__

TensorFlow 1.x selected.


'1.15.2'

In [5]:
!npx degit google-research/pegasus -f

npx: installed 1 in 2.236s
> destination directory is not empty. Using --force, continuing
> cloned google-research/pegasus#master


In [5]:
!pip install mock rouge-score sacrebleu sentencepiece tensorflow-text==1.15.1

     |████████████████████████████████| 61kB 4.0MB/s 
     |████████████████████████████████| 1.1MB 10.2MB/s 
     |████████████████████████████████| 9.1MB 19.8MB/s 
     |████████████████████████████████| 17.1MB 199kB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=02afff4a5cecc26e997472bfcae1bbd307073f0f67b225b312d4199b6e5010e3
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3


In [7]:
!mkdir ckpt
!gsutil cp -r gs://pegasus_ckpt/ ckpt/

Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.model...
Copying gs://pegasus_ckpt/c4.unigram.newline.10pct.96000.vocab...
Copying gs://pegasus_ckpt/checkpoint...
Copying gs://pegasus_ckpt/model.ckpt-1500000.data-00000-of-00001...
\ [4 files][  2.1 GiB/  2.1 GiB]   54.3 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://pegasus_ckpt/model.ckpt-1500000.index...
Copying gs://pegasus_ckpt/model.ckpt-1500000.meta...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.data-00000-of-00001...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.index...
Copying gs://pegasus_ckpt/aeslc/model.ckpt-32000.meta...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.data-00000-of-00001...
Copying gs://pegasus_ckpt/arxiv/model.ckpt-340000.index...
C

In [6]:
import os
os.environ['PYTHONPATH'] += ':/content/drive/My Drive/Text_Summarization_Code'

In [9]:
# fine-tune on aeslc
#!python3 pegasus/bin/train.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model \
--train_init_checkpoint=ckpt/pegasus_ckpt/model.ckpt-1500000 \
--model_dir=ckpt/pegasus_ckpt/aeslc

Streaming output truncated to the last 5000 lines.
I0722 09:36:16.812215 140158757775232 tpu_estimator.py:2307] global_step/sec: 0.663393
INFO:tensorflow:examples/sec: 5.30715
I0722 09:36:16.812484 140158757775232 tpu_estimator.py:2308] examples/sec: 5.30715
INFO:tensorflow:global_step/sec: 0.668461
I0722 09:36:18.308186 140158757775232 tpu_estimator.py:2307] global_step/sec: 0.668461
INFO:tensorflow:examples/sec: 5.34769
I0722 09:36:18.308770 140158757775232 tpu_estimator.py:2308] examples/sec: 5.34769
INFO:tensorflow:global_step/sec: 0.664703
I0722 09:36:19.812645 140158757775232 tpu_estimator.py:2307] global_step/sec: 0.664703
INFO:tensorflow:examples/sec: 5.31763
I0722 09:36:19.813181 140158757775232 tpu_estimator.py:2308] examples/sec: 5.31763
INFO:tensorflow:global_step/sec: 0.669365
I0722 09:36:21.306606 140158757775232 tpu_estimator.py:2307] global_step/sec: 0.669365
INFO:tensorflow:examples/sec: 5.35492
I0722 09:36:21.306870 140158757775232 tpu_estimator.py:2308] examples/sec:

In [7]:
!python3 pegasus/bin/evaluate.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/aeslc




Instructions for updating:
Use standard file APIs to check for files with this prefix.
W0723 00:31:42.736490 140588494256000 deprecation.py:323] From pegasus/bin/evaluate.py:85: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.

W0723 00:31:43.702294 140588494256000 module_wrapper.py:139] From /content/drive/My Drive/Text_Summarization_Code/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0723 00:31:45.713032 140588494256000 module_wrapper.py:139] From /content/drive/My Drive/Text_Summarization_Code/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0723 00:31:45.713606 140588494256000 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7f

In [8]:
!python3 pegasus/bin/continuous_eval.py --params=aeslc_transformer \
--param_overrides=vocab_filename=ckpt/pegasus_ckpt/c4.unigram.newline.10pct.96000.model,batch_size=1,beam_size=5,beam_alpha=0.6 \
--model_dir=ckpt/pegasus_ckpt/aeslc





W0723 01:18:05.595011 140426028275584 module_wrapper.py:139] From /content/drive/My Drive/Text_Summarization_Code/pegasus/ops/public_parsing_ops.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0723 01:18:05.754164 140426028275584 module_wrapper.py:139] From /content/drive/My Drive/Text_Summarization_Code/pegasus/params/estimator_utils.py:49: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W0723 01:18:05.754681 140426028275584 estimator.py:1994] Estimator's model_fn (<function _estimator_model_fn.<locals>.model_fn at 0x7fb708a762f0>) includes params argument, but params are not passed to Estimator.
INFO:tensorflow:Using config: {'_model_dir': 'ckpt/pegasus_ckpt/aeslc', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_ste